In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import time
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data_gene_ex = pd.read_csv("data-TCGA.csv")

In [3]:
df_gene = pd.DataFrame(data_gene_ex)

In [4]:
data_label = pd.read_csv("labels-TCGA.csv")

In [5]:
df_label = pd.DataFrame(data_label)

In [6]:
df_label.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
df_label['Class'].replace({'BRCA':0, 'KIRC':1, "LUAD":2, "PRAD":3, "COAD":4}, inplace=True)

In [8]:
df_gene1 = df_gene.copy()
df_gene1 = df_gene1.drop(columns=['Unnamed: 0'])

In [9]:
df_gene1.describe()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
count,801.000000,801.000000,801.000000,801.000000,801.000000,801.0,801.000000,801.000000,801.000000,801.000000,...,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000
mean,0.026642,3.010909,3.095350,6.722305,9.813612,0.0,7.405509,0.499882,0.016744,0.013428,...,5.896573,8.765891,10.056252,4.847727,9.741987,11.742228,10.155271,9.590726,5.528177,0.095411
std,0.136850,1.200828,1.065601,0.638819,0.506537,0.0,1.108237,0.508799,0.133635,0.204722,...,0.746399,0.603176,0.379278,2.382728,0.533898,0.670371,0.580569,0.563849,2.073859,0.364529
min,0.000000,0.000000,0.000000,5.009284,8.435999,0.0,3.930747,0.000000,0.000000,0.000000,...,2.853517,6.678368,8.669456,0.000000,7.974942,9.045255,7.530141,7.864533,0.593975,0.000000
25%,0.000000,2.299039,2.390365,6.303346,9.464466,0.0,6.676042,0.000000,0.000000,0.000000,...,5.454926,8.383834,9.826027,3.130750,9.400747,11.315857,9.836525,9.244219,4.092385,0.000000
50%,0.000000,3.143687,3.127006,6.655893,9.791599,0.0,7.450114,0.443076,0.000000,0.000000,...,5.972582,8.784144,10.066385,5.444935,9.784524,11.749802,10.191207,9.566511,5.218618,0.000000
75%,0.000000,3.883484,3.802534,7.038447,10.142324,0.0,8.121984,0.789354,0.000000,0.000000,...,6.411292,9.147136,10.299025,6.637412,10.082269,12.177852,10.578561,9.917888,6.876382,0.000000
max,1.482332,6.237034,6.063484,10.129528,11.355621,0.0,10.718190,2.779008,1.785592,4.067604,...,7.771054,11.105431,11.318243,9.207495,11.811632,13.715361,11.675653,12.813320,11.205836,5.254133


In [10]:
df_gene1.shape

(801, 20531)

In [11]:
df_gene2 = df_gene1.copy()
df_gene2 = df_gene2.T.drop_duplicates().T

In [12]:
df_gene2.shape

(801, 20265)

In [13]:
df_gene2.describe()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
count,801.000000,801.000000,801.000000,801.000000,801.000000,801.0,801.000000,801.000000,801.000000,801.000000,...,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000
mean,0.026642,3.010909,3.095350,6.722305,9.813612,0.0,7.405509,0.499882,0.016744,0.013428,...,5.896573,8.765891,10.056252,4.847727,9.741987,11.742228,10.155271,9.590726,5.528177,0.095411
std,0.136850,1.200828,1.065601,0.638819,0.506537,0.0,1.108237,0.508799,0.133635,0.204722,...,0.746399,0.603176,0.379278,2.382728,0.533898,0.670371,0.580569,0.563849,2.073859,0.364529
min,0.000000,0.000000,0.000000,5.009284,8.435999,0.0,3.930747,0.000000,0.000000,0.000000,...,2.853517,6.678368,8.669456,0.000000,7.974942,9.045255,7.530141,7.864533,0.593975,0.000000
25%,0.000000,2.299039,2.390365,6.303346,9.464466,0.0,6.676042,0.000000,0.000000,0.000000,...,5.454926,8.383834,9.826027,3.130750,9.400747,11.315857,9.836525,9.244219,4.092385,0.000000
50%,0.000000,3.143687,3.127006,6.655893,9.791599,0.0,7.450114,0.443076,0.000000,0.000000,...,5.972582,8.784144,10.066385,5.444935,9.784524,11.749802,10.191207,9.566511,5.218618,0.000000
75%,0.000000,3.883484,3.802534,7.038447,10.142324,0.0,8.121984,0.789354,0.000000,0.000000,...,6.411292,9.147136,10.299025,6.637412,10.082269,12.177852,10.578561,9.917888,6.876382,0.000000
max,1.482332,6.237034,6.063484,10.129528,11.355621,0.0,10.718190,2.779008,1.785592,4.067604,...,7.771054,11.105431,11.318243,9.207495,11.811632,13.715361,11.675653,12.813320,11.205836,5.254133


In [14]:
df_gene2.head(10)

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.000000,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.000000,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.000000,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.000000,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.000000,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0
5,0.0,3.467853,3.581918,6.620243,9.706829,0.0,7.758510,0.000000,0.0,0.000000,...,5.726657,8.602588,9.928339,6.096154,9.816001,11.556995,9.244150,9.836473,5.355133,0.0
6,0.0,1.224966,1.691177,6.572007,9.640511,0.0,6.754888,0.531868,0.0,0.000000,...,5.105904,7.927968,9.673966,1.877744,9.802692,13.256060,9.664486,9.244219,8.330912,0.0
7,0.0,2.854853,1.750478,7.226720,9.758691,0.0,5.952103,0.000000,0.0,0.000000,...,5.297833,8.277092,9.599230,5.244290,9.994339,12.670377,9.987733,9.216872,6.551490,0.0
8,0.0,3.992125,2.772730,6.546692,10.488252,0.0,7.690222,0.352307,0.0,4.067604,...,6.721974,9.597533,9.763753,7.933278,10.952880,12.498919,10.389954,10.390255,7.828321,0.0
9,0.0,3.642494,4.423558,6.849511,9.464466,0.0,7.947216,0.724214,0.0,0.000000,...,6.020051,8.712809,10.259096,6.131583,9.923582,11.144295,9.244851,9.484299,4.759151,0.0


In [15]:
x = df_gene2
y = df_label.values.reshape(-1,1)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [17]:
# feature scaling
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_scaled = scaler.transform(x)

# Removing Low Variance Features

In [19]:
# Calculate variance for each gene
gene_variances_sorted = x_train.var(axis=0).sort_values(ascending=False)

In [20]:
# Keep top N genes
N = 2000
top_genes = gene_variances_sorted.index[:N]
x_train_top_var = x_train[top_genes]
x_test_top_var = x_test[top_genes]

x_new = x.copy()
x_new = x_new[top_genes]

print(f"Shape after keeping top {N} genes:", x_train_top_var.shape)
print(f"Shape after keeping top {N} genes:", x_test_top_var.shape)

Shape after keeping top 2000 genes: (640, 2000)
Shape after keeping top 2000 genes: (161, 2000)


# ANOVA

In [21]:
# Feature selection
selector = SelectKBest(f_classif, k=1000)
x_train_sel = selector.fit_transform(x_train_top_var, y_train)
x_test_sel = selector.transform(x_test_top_var)
x_sel = selector.transform(x_new)

C:\Users\AVIDEH\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
# Get the selected feature names using the column names from x_train_top_var
selected_features = x_train_top_var.columns[selector.get_support()]

# Get F-scores and p-values
scores = selector.scores_[selector.get_support()]
pvalues = selector.pvalues_[selector.get_support()]

# Create DataFrame
selected_features_df = pd.DataFrame({
    'Feature': selected_features,
    'F-score': scores,
    'p-value': pvalues
}).sort_values(by='F-score', ascending=False).reset_index(drop=True)

# Display the result
selected_features_df.head(30)

,Feature,F-score,p-value
0,gene_9175,3308.613681,0.000000e+00
1,gene_9176,2707.862031,0.000000e+00
2,gene_220,2628.402288,0.000000e+00
3,gene_219,2551.396734,0.000000e+00
4,gene_18135,2199.089028,0.000000e+00
5,gene_13818,1862.906919,0.000000e+00
6,gene_450,1716.073131,0.000000e+00
7,gene_14114,1698.497151,0.000000e+00
8,gene_3737,1667.851209,0.000000e+00
9,gene_7964,1638.269396,0.000000e+00


In [23]:
# Get the selected feature names
selected_feature_names = x_train_top_var.columns[selector.get_support()]

# Create DataFrames with selected features and their actual values
x_train_selected_df = pd.DataFrame(x_train_sel, columns=selected_feature_names)
x_test_selected_df = pd.DataFrame(x_test_sel, columns=selected_feature_names, index=x_test.index)
x_selected_df = pd.DataFrame(x_sel, columns=selected_feature_names, index=x.index)

x_test_selected_df

,gene_9176,gene_9175,gene_15898,gene_15301,gene_15589,gene_3540,gene_19661,gene_3541,gene_11250,gene_15897,...,gene_10106,gene_11772,gene_7998,gene_9564,gene_7474,gene_13152,gene_1059,gene_2572,gene_8906,gene_16450
697,0.000000,0.000000,19.557821,12.295416,0.000000,10.315229,1.966947,17.607518,8.492262,13.902987,...,0.000000,8.648354,14.415082,10.780622,0.000000,6.769018,0.000000,3.919464,5.759787,5.397361
668,0.404576,2.380868,0.404576,0.000000,15.367524,3.132495,10.971436,4.696395,1.556993,2.288299,...,0.000000,8.902378,8.544149,8.955086,5.461267,3.546351,0.000000,3.078234,7.512693,3.585780
63,1.078815,0.000000,19.807117,0.000000,0.000000,7.118225,13.151083,15.131552,5.958358,12.240374,...,1.078815,4.903086,14.716557,10.019188,0.000000,4.468231,0.000000,7.022723,7.492558,3.599734
534,0.000000,0.000000,1.896194,11.665056,2.345822,1.708673,9.608575,0.931607,1.708673,1.493084,...,4.672476,5.590569,6.694058,10.503826,1.896194,8.072165,5.053728,5.272725,10.921424,3.869042
66,0.000000,0.000000,0.744333,0.000000,3.385859,6.346090,14.759191,8.046229,0.744333,6.246967,...,0.000000,7.367004,6.618115,13.558074,7.441749,7.312892,0.744333,2.823138,6.390542,3.476161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,18.166516,16.418269,2.577175,13.105047,0.000000,6.620104,3.687374,6.120070,14.213750,6.810919,...,4.313050,4.035615,8.630922,12.041012,4.545641,4.745888,0.000000,7.745587,6.962618,4.990946
745,0.924100,0.000000,1.697640,0.000000,0.000000,15.754655,12.142551,13.938348,0.000000,3.032048,...,0.000000,9.981068,6.337611,3.556993,0.534759,0.924100,0.000000,0.000000,10.430641,2.864394
513,0.000000,4.221452,12.067323,11.769387,0.000000,15.385276,2.868924,15.945064,7.742242,8.588658,...,1.176961,8.071301,10.746430,10.322998,1.176961,6.996999,0.000000,7.754928,7.989480,5.255520
671,2.396954,1.351402,1.113500,0.000000,13.158630,5.461286,12.752947,5.716728,0.000000,1.555571,...,0.472904,7.452579,7.430043,10.538975,4.668885,6.489101,0.000000,5.594653,6.678565,3.310573


In [25]:

# svm model with different kernels and degree = 3 and gamma= 0.7

def svm(x_tr, x_te):
    
    for k in ['linear', 'rbf', 'poly']:
        
        svm_model = SVC(kernel=k, degree=3, gamma=0.7)
        start = time.time()
        svm_model.fit(x_tr, y_train.ravel())
        train_time = time.time() - start

        y_svm_pred = svm_model.predict(x_te)
        acc = accuracy_score(y_test, y_svm_pred)

        
        print(f"{k}, accuracy = {acc:.4f}, training_time = {train_time:.4f} sec")

        

In [26]:
svm(x_train_selected_df,x_test_selected_df)

linear, accuracy = 1.0000, training_time = 1.0224 sec
rbf, accuracy = 0.3789, training_time = 0.6201 sec
poly, accuracy = 1.0000, training_time = 0.0701 sec


In [30]:
svm_model1 = SVC(kernel='poly', degree=3, gamma=0.7)

svm_model1.fit(x_train_selected_df, y_train.ravel())

y_svm_pred1 = svm_model1.predict(x_test_selected_df)

In [31]:
# Report of svm with kernel = ploy and degree=3 and gamma=0.7
print(classification_report(y_test, svm_model1.predict(x_test_selected_df)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        61
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        29
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        17

    accuracy                           1.00       161
   macro avg       1.00      1.00      1.00       161
weighted avg       1.00      1.00      1.00       161



In [32]:
# Report of svm with kernel = linear and degree=3 and gamma=0.7

svm_model2 = SVC(kernel='poly', degree=3, gamma=0.7)

svm_model2.fit(x_train_selected_df, y_train.ravel())

y_svm_pred2 = svm_model2.predict(x_test_selected_df)


print(classification_report(y_test, svm_model2.predict(x_test_selected_df)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        61
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        29
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        17

    accuracy                           1.00       161
   macro avg       1.00      1.00      1.00       161
weighted avg       1.00      1.00      1.00       161



In [33]:
# Report of svm with kernel = linear and degree=3 and gamma=0.7

svm_model3 = SVC(kernel='rbf', degree=3, gamma=0.7)

svm_model3.fit(x_train_selected_df, y_train.ravel())

y_svm_pred3 = svm_model3.predict(x_test_selected_df)


print(classification_report(y_test, svm_model3.predict(x_test_selected_df)))

              precision    recall  f1-score   support

           0       0.38      1.00      0.55        61
           1       0.00      0.00      0.00        25
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        29
           4       0.00      0.00      0.00        17

    accuracy                           0.38       161
   macro avg       0.08      0.20      0.11       161
weighted avg       0.14      0.38      0.21       161



C:\Users\AVIDEH\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AVIDEH\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AVIDEH\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
def svm (x_tr, x_te):

      for k in ['linear', 'rbf', 'poly']:

        svm_model = SVC(kernel= k)

        start = time.time()

        svm_model.fit(x_tr, y_train.ravel())
        train_time = time.time() - start

        y_svm_pred = svm_model.predict(x_te)

        acc = accuracy_score(y_test, y_svm_pred)

        
        print(f"{k}, accuracy = {acc:.4f}, training_time = {train_time:.4f} sec")

        

svm(x_train_selected_df,x_test_selected_df)

linear, accuracy = 1.0000, training_time = 0.0698 sec
rbf, accuracy = 1.0000, training_time = 0.0897 sec
poly, accuracy = 1.0000, training_time = 0.0599 sec


In [34]:
# Report of svm with kernel = linear 

svm_model4 = SVC(kernel='linear')

svm_model4.fit(x_train_selected_df, y_train.ravel())

y_svm_pred4 = svm_model4.predict(x_test_selected_df)


print(classification_report(y_test, svm_model4.predict(x_test_selected_df)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        61
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        29
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        17

    accuracy                           1.00       161
   macro avg       1.00      1.00      1.00       161
weighted avg       1.00      1.00      1.00       161



In [35]:
# Report of svm with kernel = rbf

svm_model5 = SVC(kernel='rbf')

svm_model5.fit(x_train_selected_df, y_train.ravel())

y_svm_pred5 = svm_model5.predict(x_test_selected_df)


print(classification_report(y_test, svm_model5.predict(x_test_selected_df)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        61
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        29
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        17

    accuracy                           1.00       161
   macro avg       1.00      1.00      1.00       161
weighted avg       1.00      1.00      1.00       161



In [36]:
# Report of svm with kernel = poly 

svm_model6 = SVC(kernel='poly')

svm_model6.fit(x_train_selected_df, y_train.ravel())

y_svm_pred6 = svm_model6.predict(x_test_selected_df)


print(classification_report(y_test, svm_model6.predict(x_test_selected_df)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        61
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        29
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        17

    accuracy                           1.00       161
   macro avg       1.00      1.00      1.00       161
weighted avg       1.00      1.00      1.00       161



So, we can see that different svm models ,except for one, can classify this dataset to all different cancer types with the accuracy of 1.0.
Moreover, the exception is svm with rbf kernel and degree=3 and when gamma=0.7.